![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2F05+-+TensorFlow&file=05a+-+Vertex+AI+Custom+Model+-+TensorFlow+-+Custom+Job+With+Python+File.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/05%20-%20TensorFlow/05a%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20Custom%20Job%20With%20Python%20File.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2F05%2520-%2520TensorFlow%2F05a%2520-%2520Vertex%2520AI%2520Custom%2520Model%2520-%2520TensorFlow%2520-%2520Custom%2520Job%2520With%2520Python%2520File.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/05%20-%20TensorFlow/05a%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20Custom%20Job%20With%20Python%20File.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/05%20-%20TensorFlow/05a%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20Custom%20Job%20With%20Python%20File.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

---
---
**Changes In Progress**

Every attempt is being made to ensure the public version of this notebook runs without error while the follow enhancements are being made:
- The workflow of the notebook is being adapted to a Kubeflow Pipeline running on Vertex AI Pipelines
- Including Evaluation data within Vertex AI Model Registry
- Update the Vertex AI Experiments integration which has been great simplified within the API over the past few months
- add client library reference links to each section

Order: notebook 05 and 05a will be updated first.  Then 05b-05i will follow quickly.

This note will be removed once these changes are complete.

---
---

# 05a - Vertex AI Custom Model - TensorFlow - Custom Job With Python File

**05 Series Overview**

>**NOTE:** The notebooks in the `05 - TensorFlow` series demonstrate training, serving and operations for TensorFlow models and take advantage of [Vertex AI TensorBoard](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview) to track training across experiments.  Running these notebooks will create a Vertex AI TensorBoard instance which previously (before August 2023) had a subscription cost but is now priced based on storage of which this notebook will create minimal size (<2MB). - [Vertex AI Pricing](https://cloud.google.com/vertex-ai/pricing#tensorboard).

Where a model gets trained is where it consumes computing resources.  With Vertex AI, you have choices for configuring the computing resources available at training.  This notebook is an example of an execution environment.  When it was set up there were choices for machine type and accelerators (GPUs).  

In the [05 - Vertex AI Custom Model - TensorFlow - in Notebook](./05%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20in%20Notebook.ipynb) notebook, the model training happened directly in the notebook.  The model was then imported to Vertex AI and deployed to an endpoint for online predictions. 

In this `05a-05i` series of demonstrations, the same model is trained using managed computing resources in Vertex AI Training as managed jobs.  These jobs will be demonstrated as:

-  [Custom Job](https://cloud.google.com/vertex-ai/docs/training/create-custom-job) that trains and saves (to GCS) a model from a python script (`05a`), python source distribution (`05b`), and custom container (`05c`)
-  [Training Pipeline](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview) that trains and registers a model from a python script (`05d`), python source distribution (`05e`), and custom container (`05f`)
-  [Hyperparameter Tuning Jobs](https://cloud.google.com/vertex-ai/docs/training/create-training-pipeline) from a python script (`05g`), python source distribution (`05h`), and custom container (`05i`)

**This Notebook (`05a`): An extension of `05` that runs a Custom Job using a script as an input**

This notebook trains the same Tensorflow Keras model from [05 - Vertex AI Custom Model - TensorFlow - in Notebook](./05%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20in%20Notebook.ipynb) by first modifying and saving the training code to a Python script as shown in [05 - Vertex AI Custom Model - TensorFlow - Notebook to Script](./05%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20Notebook%20to%20Script.ipynb).  

The script is then used as an input for a Vertex AI > Training > Custom Job that is also assigned compute resources and a [pre-built container for custom training](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers) for executing the training in a managed service. 

This job is launched using the Vertex AI client library:
- [Python Cloud Client Libraries](https://cloud.google.com/python/docs/reference)
    - [google-cloud-aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest)
        - [`aiplatform` package](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform)
            - [`aiplatform.CustomJob.from_local_script()`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.CustomJob#google_cloud_aiplatform_CustomJob_from_local_script)


|Custom Job|Workflow With Custom Job|
|:---:|:---:|
|![](../architectures/architectures/images/custom%20training/a_job.png)|![](../architectures/architectures/images/custom%20training/a_workflow.png)|

**Vertex AI Training**

In Vertex AI Training you can run your training code as a job where you specify the compute resources to use. For tips on preparing code, running training jobs, and workflows for building custom containers with software and training code combined please visit these [tips notebooks](../Tips/readme.md) in this repository:
- [Python Packages](../Tips/Python%20Packages.ipynb)
- [Python Custom Containers](../Tips/Python%20Custom%20Containers.ipynb)
- [Python Training](../Tips/Python%20Training.ipynb)
- [Python Job Parameters](../Tips/Python%20Job%20Parameters.ipybnd)

<p align="center" width="100%">
    <img src="../architectures/overview/training.png" width="45%">
    &nbsp; &nbsp; &nbsp; &nbsp;
    <img src="../architectures/overview/training2.png" width="45%">
</p>

**Prerequisites:**
-  [01 - BigQuery - Table Data Source](../01%20-%20Data%20Sources/01%20-%20BigQuery%20-%20Table%20Data%20Source.ipynb)
-  Understanding:
    -  Model overview in [05 - Vertex AI Custom Model - TensorFlow - in Notebook](./05%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20in%20Notebook.ipynb)
    -  Convert notebook code to Python Script in [05 - Vertex AI Custom Model - TensorFlow - Notebook to Script](./05%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20Notebook%20to%20Script.ipynb)

**Resources:**
-  [BigQuery Tensorflow Reader](https://www.tensorflow.org/io/tutorials/bigquery)
-  [Keras Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)
   -  [Keras API](https://www.tensorflow.org/api_docs/python/tf/keras)
-  [Python Client For Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html)
-  [Tensorflow Python Client](https://www.tensorflow.org/api_docs/python/tf)
-  [Tensorflow I/O Python Client](https://www.tensorflow.org/io/api_docs/python/tfio/bigquery)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)
-  Containers for training (Pre-Built)
   -  [Overview](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container)
    - Pre-built Containers for Vertex AI
        - [Training](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)
        - [Prediction & Explaination](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)

<!--
**Conceptual Flow & Workflow**

<p align="center">
  <img alt="Conceptual Flow" src="../architectures/slides/05a_arch.png" width="45%">
&nbsp; &nbsp; &nbsp; &nbsp;
  <img alt="Workflow" src="../architectures/slides/05a_console.png" width="45%">
</p>
-->

---
## Colab Setup

To run this notebook in Colab click [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/05%20-%20TensorFlow/05a%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20Custom%20Job%20With%20Python%20File.ipynb) and run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    import google.colab
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [3]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('kfp', 'kfp'),
    ('google_cloud_pipeline_components', 'google-cloud-pipeline-components')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [4]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

inputs:

In [5]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [6]:
REGION = 'us-central1'
EXPERIMENT = '05a'
SERIES = '05'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud_prepped'

# specify a GCS Bucket
GCS_BUCKET = PROJECT_ID

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id,splits' # add more variables to the string with comma delimiters
EPOCHS = 10
BATCH_SIZE = 100

packages:

In [7]:
from google.cloud import aiplatform
from datetime import datetime
import importlib
from IPython.display import Markdown as md
from google.cloud import bigquery
from google.cloud import storage
#from google.protobuf import json_format
#from google.protobuf.struct_pb2 import Value
import json
import os
import numpy as np
import pandas as pd

clients:

In [8]:
aiplatform.init(project = PROJECT_ID, location = REGION)
bq = bigquery.Client(project = PROJECT_ID)
gcs = storage.Client(project = PROJECT_ID)

parameters:

In [9]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
URI = f"gs://{GCS_BUCKET}/{SERIES}/{EXPERIMENT}"
DIR = f"temp/{EXPERIMENT}"

In [10]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

List the service accounts current roles:

In [11]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

ROLE
roles/bigquery.admin
roles/owner
roles/run.admin
roles/secretmanager.secretAccessor
roles/storage.objectAdmin


>Note: If the resulting list is missing [roles/storage.objectAdmin](https://cloud.google.com/storage/docs/access-control/iam-roles) then [revisit the setup notebook](../00%20-%20Setup/00%20-%20Environment%20Setup.ipynb#permissions) and add this permission to the service account with the provided instructions.

environment:

In [12]:
if not os.path.exists(DIR):
    os.makedirs(DIR)

Experiment Tracking:

In [13]:
FRAMEWORK = 'tf'
TASK = 'classification'
MODEL_TYPE = 'dnn'
EXPERIMENT_NAME = f'experiment-{SERIES}-{EXPERIMENT}-{FRAMEWORK}-{TASK}-{MODEL_TYPE}'
RUN_NAME = f'run-{TIMESTAMP}'

---
## Get Vertex AI Experiments Tensorboard Instance Name
[Vertex AI Experiments](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview) has managed [Tensorboard](https://www.tensorflow.org/tensorboard) instances that you can track Tensorboard Experiments (a training run or hyperparameter tuning sweep).  

The training job will show up as an experiment for the Tensorboard instance and have the same name as the training job ID.

This code checks to see if a Tensorboard Instance has been created in the project, retrieves it if so, creates it otherwise:

In [14]:
tb = aiplatform.Tensorboard.list(filter=f"labels.series={SERIES}")
if tb:
    tb = tb[0]
else: 
    tb = aiplatform.Tensorboard.create(display_name = SERIES, labels = {'series' : f'{SERIES}'})

In [15]:
tb.resource_name

'projects/1026793852137/locations/us-central1/tensorboards/7876136041294331904'

---
## Setup Vertex AI Experiments

The code in this section initializes the experiment and starts a run that represents this notebook.  Throughout the notebook sections for model training and evaluation information will be logged to the experiment using:
- [.log_params](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_params)
- [.log_metrics](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_metrics)
- [.log_time_series_metrics](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_time_series_metrics)

In [16]:
aiplatform.init(experiment = EXPERIMENT_NAME, experiment_tensorboard = tb.resource_name)

---
## Training

### Python File for Training

This notebook trains the same Tensorflow Keras model from [05 - Vertex AI Custom Model - TensorFlow - in Notebook](./05%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20in%20Notebook.ipynb) by first modifying and saving the training code to a python script as shown in [05 - Vertex AI Custom Model - TensorFlow - Notebook to Script](./05%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20Notebook%20to%20Script.ipynb) which stores the script in [`./code/train.py`](./code/train.py).

Retrieve the training script (if not already included in a clone of this repository):

In [17]:
file = "./code/train.py"
if not os.path.exists(file):
    print('Retrieving document...')
    if not os.path.exists(os.path.dirname(file)):
      os.makedirs(os.path.dirname(file))
    import requests, urllib.parse
    r = requests.get(f'https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/05%20-%20TensorFlow/{urllib.parse.quote(file[2:])}')
    open(file, 'wb').write(r.content)
    print(f'Document now at `{file}`')
else:
    print(f'Document Found at `{file}`')

Document Found at `./code/train.py`


**Review the script:**

In [18]:
SCRIPT_PATH = './code/train.py'

with open(SCRIPT_PATH, 'r') as file:
    data = file.read()
md(f"```python\n\n{data}\n```")

```python


# package import
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
import tensorflow as tf
from google.cloud import bigquery
from google.cloud import aiplatform
import argparse
import os

# import argument to local variables
parser = argparse.ArgumentParser()
# the passed param, dest: a name for the param, default: if absent fetch this param from the OS, type: type to convert to, help: description of argument
parser.add_argument('--epochs', dest = 'epochs', default = 10, type = int, help = 'Number of Epochs')
parser.add_argument('--batch_size', dest = 'batch_size', default = 32, type = int, help = 'Batch Size')
parser.add_argument('--var_target', dest = 'var_target', type=str)
parser.add_argument('--var_omit', dest = 'var_omit', type=str)#, nargs='*')
parser.add_argument('--project_id', dest = 'project_id', type=str)
parser.add_argument('--bq_project', dest = 'bq_project', type=str)
parser.add_argument('--bq_dataset', dest = 'bq_dataset', type=str)
parser.add_argument('--bq_table', dest = 'bq_table', type=str)
parser.add_argument('--region', dest = 'region', type=str)
parser.add_argument('--experiment', dest = 'experiment', type=str)
parser.add_argument('--series', dest = 'series', type=str)
parser.add_argument('--experiment_name', dest = 'experiment_name', type=str)
parser.add_argument('--run_name', dest = 'run_name', type=str)
args = parser.parse_args()

# clients
bq = bigquery.Client(project = args.project_id)
aiplatform.init(project = args.project_id, location = args.region)

# Vertex AI Experiment
if args.run_name in [run.name for run in aiplatform.ExperimentRun.list(experiment = args.experiment_name)]:
    expRun = aiplatform.ExperimentRun(run_name = args.run_name, experiment = args.experiment_name)
else:
    expRun = aiplatform.ExperimentRun.create(run_name = args.run_name, experiment = args.experiment_name)
expRun.log_params({'experiment': args.experiment, 'series': args.series, 'project_id': args.project_id})

# get schema from bigquery source
query = f"SELECT * FROM {args.bq_project}.{args.bq_dataset}.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{args.bq_table}'"
schema = bq.query(query).to_dataframe()

# get number of classes from bigquery source
nclasses = bq.query(query = f'SELECT DISTINCT {args.var_target} FROM {args.bq_project}.{args.bq_dataset}.{args.bq_table} WHERE {args.var_target} is not null').to_dataframe()
nclasses = nclasses.shape[0]
expRun.log_params({'data_source': f'bq://{args.bq_project}.{args.bq_dataset}.{args.bq_table}', 'nclasses': nclasses, 'var_split': 'splits', 'var_target': args.var_target})

# Make a list of columns to omit
OMIT = [x for x in args.var_omit.split(',') if x != '']

# use schema to prepare a list of columns to read from BigQuery
selected_fields = schema[~schema.column_name.isin(OMIT)].column_name.tolist()

# all the columns in this data source are either float64 or int64
output_types = [dtypes.float64 if x=='FLOAT64' else dtypes.int64 for x in schema[~schema.column_name.isin(OMIT)].data_type.tolist()]

# remap input data to Tensorflow inputs of features and target
def transTable(row_dict):
    target = row_dict.pop(args.var_target)
    target = tf.one_hot(tf.cast(target, tf.int64), nclasses)
    target = tf.cast(target, tf.float32)
    return(row_dict, target)

# function to setup a bigquery reader with Tensorflow I/O
def bq_reader(split):
    reader = BigQueryClient()

    training = reader.read_session(
        parent = f"projects/{args.project_id}",
        project_id = args.bq_project,
        table_id = args.bq_table,
        dataset_id = args.bq_dataset,
        selected_fields = selected_fields,
        output_types = output_types,
        row_restriction = f"splits='{split}'",
        requested_streams = 3
    )
    
    return training

# setup feed for train, validate and test
train = bq_reader('TRAIN').parallel_read_rows().prefetch(1).map(transTable).shuffle(args.batch_size*10).batch(args.batch_size)
validate = bq_reader('VALIDATE').parallel_read_rows().prefetch(1).map(transTable).batch(args.batch_size)
test = bq_reader('TEST').parallel_read_rows().prefetch(1).map(transTable).batch(args.batch_size)
expRun.log_params({'training.batch_size': args.batch_size, 'training.shuffle': 10*args.batch_size, 'training.prefetch': 1})
# Logistic Regression

# feature list
numeric_features = [feature for feature in schema[~schema.column_name.isin(OMIT + [args.var_target])]['column_name'].to_list()]

# feature inputs
features = [tf.keras.Input(shape = (1,), dtype = dtypes.float64, name = feature) for feature in numeric_features]

# normalize features - before training
#normalized_features = []
#for feature in features:
#    normalizer = tf.keras.layers.Normalization(axis = None, name = feature.name + '_normalized')
#    feature_data = train.map(lambda x, y: x[feature.name])
#    normalizer.adapt(feature_data)
#    normalized_features.append(normalizer(feature))

# concatenate features
all_features = tf.keras.layers.Concatenate(name = 'feature_layer')(features)
#all_features = tf.keras.layers.Concatenate(name = 'feature_layer')(normalized_features) # (features)

# batch normalization of inputs - during training
all_features = tf.keras.layers.BatchNormalization(name = 'batch_normalization_layer')(all_features)

# logistic - using softmax activation to nclasses
logistic = tf.keras.layers.Dense(nclasses, activation = tf.nn.softmax, name = 'logistic')(all_features)

# the model
model = tf.keras.Model(
    inputs = features,
    outputs = logistic,
    name = args.experiment
)

# compile the model
model.compile(
    optimizer = tf.keras.optimizers.SGD(), #SGD or Adam
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = ['accuracy', tf.keras.metrics.AUC(curve = 'PR', name = 'auprc')]
)

# setup tensorboard logs and train
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=os.environ['AIP_TENSORBOARD_LOG_DIR'], histogram_freq=1)
history = model.fit(train, epochs = args.epochs, callbacks = [tensorboard_callback], validation_data = validate)
expRun.log_params({'training.epochs': history.params['epochs']})
for e in range(0, history.params['epochs']):
    expRun.log_time_series_metrics(
        {
            'train_loss': history.history['loss'][e],
            'train_accuracy': history.history['accuracy'][e],
            'train_auprc': history.history['auprc'][e],
            'val_loss': history.history['val_loss'][e],
            'val_accuracy': history.history['val_accuracy'][e],
            'val_auprc': history.history['val_auprc'][e]
        }
    )

# test evaluations:
loss, accuracy, auprc = model.evaluate(test)
expRun.log_metrics({'test_loss': loss, 'test_accuracy': accuracy, 'test_auprc': auprc})

# val evaluations:
loss, accuracy, auprc = model.evaluate(validate)
expRun.log_metrics({'val_loss': loss, 'val_accuracy': accuracy, 'val_auprc': auprc})

# training evaluations:
loss, accuracy, auprc = model.evaluate(train)
expRun.log_metrics({'train_loss': loss, 'train_accuracy': accuracy, 'train_auprc': auprc})

# output the model save files
model.save(os.getenv("AIP_MODEL_DIR"))
expRun.log_params({'model.save': os.getenv("AIP_MODEL_DIR")})
expRun.end_run()

```

### Choose Computing Environment

When using [custom training on Vertex AI](https://cloud.google.com/vertex-ai/docs/training/custom-training-methods) the compute environment is specified as parameters.  At a minimum this will include the [compute resources](https://cloud.google.com/vertex-ai/docs/training/configure-compute) and [container](https://cloud.google.com/vertex-ai/docs/training/configure-container-settings) URIs.

This example uses minimal compute with a single node and no accelerators (GPU).

For a container, a [pre-built custome training container](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers) for TensorFlow is being used and additional packages are being specified in the jobs parameters. This leads to two important considerations:
- What is the Python version on the container? 
    - This will have an impact when additional packages are being installed and the packages may have version requirments.
- What is the framework version on the container?
    - Make sure the options you need from a framework are included in the version you pick.

In [19]:
# Resources
TRAIN_COMPUTE = 'n1-standard-4'
DEPLOY_COMPUTE = 'n1-standard-4'
TRAIN_IMAGE = 'us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-12.py310:latest'
DEPLOY_IMAGE ='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-12:latest'

### Setup Training Job

Run the job with [`aiplatform.CustomJob.from_local_script()`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.CustomJob#google_cloud_aiplatform_CustomJob_from_local_script).

In [20]:
CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--batch_size=" + str(BATCH_SIZE),
    "--var_target=" + VAR_TARGET,
    "--var_omit=" + VAR_OMIT,
    "--project_id=" + PROJECT_ID,
    "--bq_project=" + BQ_PROJECT,
    "--bq_dataset=" + BQ_DATASET,
    "--bq_table=" + BQ_TABLE,
    "--region=" + REGION,
    "--experiment=" + EXPERIMENT,
    "--series=" + SERIES,
    "--experiment_name=" + EXPERIMENT_NAME,
    "--run_name=" + RUN_NAME
]

In [21]:
customJob = aiplatform.CustomJob.from_local_script(
    display_name = f'{SERIES}_{EXPERIMENT}_{TIMESTAMP}',
    script_path = SCRIPT_PATH,
    container_uri = TRAIN_IMAGE,
    args = CMDARGS,
    requirements = ['tensorflow_io', f'google-cloud-aiplatform>={aiplatform.__version__}', 'db-dtypes', f"protobuf>={importlib.metadata.version('protobuf')}"],
    replica_count = 1,
    machine_type = TRAIN_COMPUTE,
    accelerator_count = 0,
    base_output_dir = f"{URI}/models/{TIMESTAMP}",
    staging_bucket = f"{URI}/models/{TIMESTAMP}",
    labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}
)

Training script copied to:
gs://statmike-mlops-349915/05/05a/models/20240222135907/aiplatform-2024-02-22-13:59:13.980-aiplatform_custom_trainer_script-0.1.tar.gz.


### Run Training Job

In [22]:
customJob.run(
    service_account = SERVICE_ACCOUNT,
    tensorboard = tb.resource_name
)

Creating CustomJob
CustomJob created. Resource name: projects/1026793852137/locations/us-central1/customJobs/5645855250712625152
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/1026793852137/locations/us-central1/customJobs/5645855250712625152')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/5645855250712625152?project=1026793852137
View Tensorboard:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7876136041294331904+experiments+5645855250712625152
CustomJob projects/1026793852137/locations/us-central1/customJobs/5645855250712625152 current state:
JobState.JOB_STATE_QUEUED
CustomJob projects/1026793852137/locations/us-central1/customJobs/5645855250712625152 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/1026793852137/locations/us-central1/customJobs/5645855250712625152 current state:
JobState.JOB_STATE_PENDING
C

In [23]:
customJob.display_name

'05_05a_20240222135907'

In [24]:
customJob.resource_name

'projects/1026793852137/locations/us-central1/customJobs/5645855250712625152'

Create hyperlinks to job and tensorboard here:

In [25]:
job_link = f"https://console.cloud.google.com/vertex-ai/locations/{REGION}/training/{customJob.resource_name.split('/')[-1]}/cpu?cloudshell=false&project={PROJECT_ID}"
board_link = f"https://{REGION}.tensorboard.googleusercontent.com/experiment/{tb.resource_name.replace('/', '+')}+experiments+{customJob.resource_name.split('/')[-1]}"

print(f'Review the Custom Job here:\n{job_link}')
print(f'Review the TensorBoard From the Job here:\n{board_link}')

Review the Custom Job here:
https://console.cloud.google.com/vertex-ai/locations/us-central1/training/5645855250712625152/cpu?cloudshell=false&project=statmike-mlops-349915
Review the TensorBoard From the Job here:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7876136041294331904+experiments+5645855250712625152


---
## Serving

### Upload The Model

In [26]:
modelmatch = aiplatform.Model.list(filter = f'display_name={SERIES}_{EXPERIMENT} AND labels.series={SERIES} AND labels.experiment={EXPERIMENT}')

upload_model = True
if modelmatch:
    print("Model Already in Registry:")
    if RUN_NAME in modelmatch[0].version_aliases:
        print("This version already loaded, no action taken.")
        upload_model = False
        model = aiplatform.Model(model_name = modelmatch[0].resource_name)
    else:
        print('Loading model as new default version.')
        parent_model = modelmatch[0].resource_name

else:
    print('This is a new model, creating in model registry')
    parent_model = ''

if upload_model:
    model = aiplatform.Model.upload(
        display_name = f'{SERIES}_{EXPERIMENT}',
        model_id = f'model_{SERIES}_{EXPERIMENT}',
        parent_model =  parent_model,
        serving_container_image_uri = DEPLOY_IMAGE,
        artifact_uri = f"{URI}/models/{TIMESTAMP}/model",
        is_default_version = True,
        version_aliases = [RUN_NAME],
        version_description = RUN_NAME,
        labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}        
    )

Model Already in Registry:
Loading model as new default version.
Creating Model
Create Model backing LRO: projects/1026793852137/locations/us-central1/models/model_05_05a/operations/7205917767826931712
Model created. Resource name: projects/1026793852137/locations/us-central1/models/model_05_05a@22
To use this Model in another session:
model = aiplatform.Model('projects/1026793852137/locations/us-central1/models/model_05_05a@22')


>**Note** on Version Aliases:
>Expectation is a name starting with `a-z` that can include `[a-zA-Z0-9-]`
>
>**Retrieve a Model Resource**
>[aiplatform.Model()](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model)
>```Python
model = aiplatform.Model(model_name = f'model_{SERIES}_{EXPERIMENT}') # retrieves default version
model = aiplatform.Model(model_name = f'model_{SERIES}_{EXPERIMENT}@time-{TIMESTAMP}') # retrieves specific version
model = aiplatform.Model(model_name = f'model_{SERIES}_{EXPERIMENT}', version = f'time-{TIMESTAMP}') # retrieves specific version
```

In [27]:
print(f'Review the model in the Vertex AI Model Registry:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/models/{model.name}?project={PROJECT_ID}')

Review the model in the Vertex AI Model Registry:
https://console.cloud.google.com/vertex-ai/locations/us-central1/models/model_05_05a?project=statmike-mlops-349915


### Vertex AI Experiment Update and Review

In [28]:
expRun = aiplatform.ExperimentRun(run_name = RUN_NAME, experiment = EXPERIMENT_NAME)

In [29]:
expRun.log_params({
    'model.uri': model.uri,
    'model.display_name': model.display_name,
    'model.name': model.name,
    'model.resource_name': model.resource_name,
    'model.version_id': model.version_id,
    'model.versioned_resource_name': model.versioned_resource_name,
    'customJobs.display_name': customJob.display_name,
    'customJobs.resource_name': customJob.resource_name,
    'customJobs.link': job_link,
    'customJobs.tensorboard': board_link
})

Complete the experiment run:

In [30]:
expRun.update_state(state = aiplatform.gapic.Execution.State.COMPLETE)

Retrieve the experiment:

In [31]:
exp = aiplatform.Experiment(experiment_name = EXPERIMENT_NAME)

In [32]:
exp.get_data_frame()

,experiment_name,run_name,run_type,state,param.training.epochs,param.training.shuffle,param.customJobs.resource_name,param.model.version_id,param.customJobs.tensorboard,param.model.name,...,metric.train_loss,metric.test_accuracy,metric.train_auprc,metric.train_accuracy,time_series_metric.val_auprc,time_series_metric.train_auprc,time_series_metric.val_accuracy,time_series_metric.train_accuracy,time_series_metric.train_loss,time_series_metric.val_loss
0,experiment-05-05a-tf-classification-dnn,run-20240222135907,system.ExperimentRun,COMPLETE,10.0,1000.0,projects/1026793852137/locations/us-central1/c...,22,https://us-central1.tensorboard.googleusercont...,model_05_05a,...,0.006899,0.999263,0.999374,0.999237,0.999483,0.999453,0.999115,0.999241,0.005181,0.006963
1,experiment-05-05a-tf-classification-dnn,run-20231221182010,system.ExperimentRun,COMPLETE,10.0,1000.0,projects/1026793852137/locations/us-central1/c...,21,https://us-central1.tensorboard.googleusercont...,model_05_05a,...,0.006786,0.999123,0.999431,0.999066,0.999431,0.999560,0.999009,0.999202,0.004724,0.006656
2,experiment-05-05a-tf-classification-dnn,run-20231221175611,system.ExperimentRun,RUNNING,NaN,1000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,experiment-05-05a-tf-classification-dnn,run-20231221013301,system.ExperimentRun,COMPLETE,10.0,1000.0,projects/1026793852137/locations/us-central1/c...,20,https://us-central1.tensorboard.googleusercont...,model_05_05a,...,0.006357,0.999439,0.999430,0.999320,0.999528,0.999522,0.999079,0.999290,0.004386,0.006117
4,experiment-05-05a-tf-classification-dnn,run-20231221010255,system.ExperimentRun,COMPLETE,10.0,1000.0,projects/1026793852137/locations/us-central1/c...,19,https://us-central1.tensorboard.googleusercont...,model_05_05a,...,0.006426,0.999298,0.999371,0.999237,0.999388,0.999436,0.999186,0.999110,0.005539,0.005936
5,experiment-05-05a-tf-classification-dnn,run-20231220223853,system.ExperimentRun,RUNNING,NaN,1000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,experiment-05-05a-tf-classification-dnn,run-20231220223157,system.ExperimentRun,RUNNING,NaN,1000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,experiment-05-05a-tf-classification-dnn,run-20231220220654,system.ExperimentRun,RUNNING,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,experiment-05-05a-tf-classification-dnn,run-20231217142807,system.ExperimentRun,COMPLETE,10.0,1000.0,projects/1026793852137/locations/us-central1/c...,18,https://us-central1.tensorboard.googleusercont...,model_05_05a,...,0.005391,0.999368,0.999529,0.999180,0.999523,0.999640,0.999221,0.999189,0.004216,0.005396
9,experiment-05-05a-tf-classification-dnn,run-20231217134246,system.ExperimentRun,COMPLETE,10.0,1000.0,projects/1026793852137/locations/us-central1/c...,17,https://us-central1.tensorboard.googleusercont...,model_05_05a,...,0.005509,0.999263,0.999432,0.999228,0.999484,0.999559,0.999009,0.999158,0.004695,0.005886


Review the Experiments TensorBoard to compare runs:

In [33]:
print(f"The Experiment TensorBoard Link:\nhttps://{REGION}.tensorboard.googleusercontent.com/experiment/{tb.resource_name.replace('/', '+')}+experiments+{exp.name}")

The Experiment TensorBoard Link:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7876136041294331904+experiments+experiment-05-05a-tf-classification-dnn


In [34]:
expRun.get_time_series_data_frame()

,step,wall_time,val_auprc,train_auprc,val_accuracy,train_accuracy,train_loss,val_loss
0,1,2024-02-22 14:09:29.956000+00:00,0.998944,0.997525,0.998902,0.984982,0.058303,0.013537
1,2,2024-02-22 14:09:30.064000+00:00,0.999274,0.999278,0.999009,0.999167,0.009147,0.010164
2,3,2024-02-22 14:09:30.184000+00:00,0.999317,0.999353,0.999009,0.999255,0.007136,0.009021
3,4,2024-02-22 14:09:30.281000+00:00,0.999378,0.999386,0.999044,0.999277,0.006365,0.008441
4,5,2024-02-22 14:09:30.368000+00:00,0.999430,0.999402,0.999044,0.999285,0.006013,0.007969
5,6,2024-02-22 14:09:30.469000+00:00,0.999432,0.999389,0.999044,0.999294,0.005725,0.007686
6,7,2024-02-22 14:09:30.569000+00:00,0.999481,0.999397,0.999079,0.999259,0.005534,0.007394
7,8,2024-02-22 14:09:30.667000+00:00,0.999482,0.999387,0.999115,0.999237,0.005395,0.007210
8,9,2024-02-22 14:09:30.761000+00:00,0.999482,0.999446,0.999115,0.999233,0.005186,0.007098
9,10,2024-02-22 14:09:30.847000+00:00,0.999483,0.999453,0.999115,0.999241,0.005181,0.006963


### Review Experiment and Run in Console

In [35]:
print(f'Review The Experiment in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/experiments/{EXPERIMENT_NAME}?project={PROJECT_ID}')

Review The Experiment in the Console:
https://console.cloud.google.com/vertex-ai/locations/us-central1/experiments/experiment-05-05a-tf-classification-dnn?project=statmike-mlops-349915


In [36]:
print(f'Review The Experiment Run in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/experiments/{EXPERIMENT_NAME}/runs/{EXPERIMENT_NAME}-{RUN_NAME}?project={PROJECT_ID}')

Review The Experiment Run in the Console:
https://console.cloud.google.com/vertex-ai/locations/us-central1/experiments/experiment-05-05a-tf-classification-dnn/runs/experiment-05-05a-tf-classification-dnn-run-20240222135907?project=statmike-mlops-349915


### Compare This Run Using Experiments

Get a list of all experiments in this project:

In [37]:
experiments = aiplatform.Experiment.list()

Remove experiments not in the SERIES:

In [38]:
experiments = [e for e in experiments if e.name.split('-')[0:2] == ['experiment', SERIES]]

Combine the runs from all experiments in SERIES into a single dataframe:

In [39]:
results = []
for experiment in experiments:
        results.append(experiment.get_data_frame())
        print(experiment.name)
results = pd.concat(results)

experiment-05-05f-tf-classification-dnn
experiment-05-05f2-tf-classification-dnn
experiment-05-05i-tf-classification-dnn
experiment-05-05h-tf-classification-dnn
experiment-05-05g-tf-classification-dnn
experiment-05-05e-tf-classification-dnn
experiment-05-05d-tf-classification-dnn
experiment-05-05c-tf-classification-dnn
experiment-05-05b-tf-classification-dnn
experiment-05-05a-tf-classification-dnn
experiment-05-05-tf-classification-dnn


Create ranks for models within experiment and across the entire SERIES:

In [40]:
def ranker(metric = 'metric.test_auprc'):
    ranks = results[['experiment_name', 'run_name', 'param.model.display_name', 'param.model.version_id', metric]].copy().reset_index(drop = True)
    ranks = ranks[ranks['param.model.display_name'].notnull()]
    ranks['series_rank'] = ranks[metric].rank(method = 'dense', ascending = False)
    ranks['experiment_rank'] = ranks.groupby('experiment_name')[metric].rank(method = 'dense', ascending = False)
    return ranks.sort_values(by = ['experiment_name', 'run_name'])
    
ranks = ranker('metric.test_auprc')
ranks

,experiment_name,run_name,param.model.display_name,param.model.version_id,metric.test_auprc,series_rank,experiment_rank
173,experiment-05-05-tf-classification-dnn,run-20230210115433,05_05,6,0.999130,44.0,9.0
171,experiment-05-05-tf-classification-dnn,run-20230308225745,05_05,7,0.999469,20.0,3.0
170,experiment-05-05-tf-classification-dnn,run-20230324103811,05_05,8,0.999299,38.0,7.0
155,experiment-05-05-tf-classification-dnn,run-20230327111418,05_05,12,0.999431,23.0,5.0
147,experiment-05-05-tf-classification-dnn,run-20231220191718,05_05,14,0.999236,42.0,8.0
146,experiment-05-05-tf-classification-dnn,run-20231220203613,05_05,15,0.999511,16.0,2.0
145,experiment-05-05-tf-classification-dnn,run-20231220214331,05_05,16,0.999377,31.0,6.0
144,experiment-05-05-tf-classification-dnn,run-20231221021319,05_05,17,0.999453,21.0,4.0
143,experiment-05-05-tf-classification-dnn,run-20231221123117,05_05,18,0.999649,2.0,1.0
142,experiment-05-05a-tf-classification-dnn,run-20230210122632,05_05a,5,0.999154,43.0,17.0


In [41]:
current_rank = ranks.loc[(ranks['param.model.display_name'] == model.display_name) & (ranks['param.model.version_id'] == model.version_id)]
current_rank

,experiment_name,run_name,param.model.display_name,param.model.version_id,metric.test_auprc,series_rank,experiment_rank
119,experiment-05-05a-tf-classification-dnn,run-20240222135907,05_05a,22,0.999397,26.0,10.0


In [42]:
print(f"The current model is ranked {current_rank['experiment_rank'].iloc[0]} within this experiment and {current_rank['series_rank'].iloc[0]} across this series.")

The current model is ranked 10.0 within this experiment and 26.0 across this series.


### Create/Retrieve The Endpoint For This Series

In [43]:
endpoints = aiplatform.Endpoint.list(filter = f"labels.series={SERIES} AND display_name={SERIES}")
if endpoints:
    endpoint = endpoints[0]
    print(f"Endpoint Exists: {endpoints[0].resource_name}")
else:
    endpoint = aiplatform.Endpoint.create(
        display_name = f"{SERIES}",
        labels = {'series' : f"{SERIES}"}    
    )
    print(f"Endpoint Created: {endpoint.resource_name}")
    
print(f'Review the Endpoint in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/endpoints/{endpoint.name}?project={PROJECT_ID}')

Endpoint Exists: projects/1026793852137/locations/us-central1/endpoints/725723853820526592
Review the Endpoint in the Console:
https://console.cloud.google.com/vertex-ai/locations/us-central1/endpoints/725723853820526592?project=statmike-mlops-349915


In [44]:
endpoint.display_name

'05'

In [45]:
endpoint.traffic_split

{'2423682068408958976': 100}

In [46]:
deployed_models = endpoint.list_models()
#deployed_models

### Should This Model Be Deployed?
Is it better than the model already deployed on the endpoint?

In [47]:
deploy = False
if deployed_models:
    for deployed_model in endpoint.list_models():
        deployed_rank = ranks.loc[(ranks['param.model.display_name'] == deployed_model.display_name) & (ranks['param.model.version_id'] == deployed_model.model_version_id)]['series_rank'].iloc[0]
        model_rank = current_rank['series_rank'].iloc[0]
        if deployed_model.display_name == model.display_name and deployed_model.model_version_id == model.version_id:
            print(f'The current model/version is already deployed.')
            break
        elif model_rank <= deployed_rank:
            deploy = True
            print(f'The current model is ranked better ({model_rank}) than a currently deployed model ({deployed_rank}).')
            break
    if deploy == False: print(f'The current model is ranked worse ({model_rank}) than a currently deployed model ({deployed_rank})')
else: 
    deploy = True
    print('No models currently deployed.')

The current model is ranked worse (26.0) than a currently deployed model (2.0)


### Deploy Model To Endpoint

In [48]:
if deploy:
    print(f'Deploying model with 100% of traffic...')
    endpoint.deploy(
        model = model,
        deployed_model_display_name = model.display_name,
        traffic_percentage = 100,
        machine_type = DEPLOY_COMPUTE,
        min_replica_count = 1,
        max_replica_count = 1
    )
else: print(f'Not deploying - current model is worse ({model_rank}) than the currently deployed model ({deployed_rank})')

Not deploying - current model is worse (26.0) than the currently deployed model (2.0)


### Remove Deployed Models without Traffic

In [49]:
for deployed_model in endpoint.list_models():
    if deployed_model.id in endpoint.traffic_split:
        print(f"Model {deployed_model.display_name} with version {deployed_model.model_version_id} has traffic = {endpoint.traffic_split[deployed_model.id]}")
    else:
        endpoint.undeploy(deployed_model_id = deployed_model.id)
        print(f"Undeploying {deployed_model.display_name} with version {deployed_model.model_version_id} because it has no traffic.")

Model 05_05 with version 18 has traffic = 100


In [50]:
endpoint.traffic_split

{'2423682068408958976': 100}

In [51]:
#endpoint.list_models()

---
## Prediction

See many more details on requesting predictions in the [05Tools - Prediction](./05Tools%20-%20Prediction.ipynb) notebook.

### Prepare a record for prediction: instance and parameters lists

In [52]:
n = 10
# Make a list of columns to omit
OMIT = [x for x in VAR_OMIT.split(',') if x != '']
pred = bq.query(
    query = f"""
        SELECT * EXCEPT({','.join([VAR_TARGET] + OMIT)})
        FROM {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}
        WHERE splits='TEST'
        LIMIT {n}
        """
).to_dataframe()

In [53]:
pred

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,35337,1.092844,-0.013230,1.359829,2.731537,-0.707357,0.873837,-0.796130,0.437707,0.396770,...,-0.240428,0.037603,0.380026,-0.167647,0.027557,0.592115,0.219695,0.036970,0.010984,0.0
1,60481,1.238973,0.035226,0.063003,0.641406,-0.260893,-0.580097,0.049938,-0.034733,0.405932,...,-0.265080,-0.060003,-0.053585,-0.057718,0.104983,0.537987,0.589563,-0.046207,-0.006212,0.0
2,139587,1.870539,0.211079,0.224457,3.889486,-0.380177,0.249799,-0.577133,0.179189,-0.120462,...,-0.374356,0.196006,0.656552,0.180776,-0.060226,-0.228979,0.080827,0.009868,-0.036997,0.0
3,162908,-3.368339,-1.980442,0.153645,-0.159795,3.847169,-3.516873,-1.209398,-0.292122,0.760543,...,-0.923275,-0.545992,-0.252324,-1.171627,0.214333,-0.159652,-0.060883,1.294977,0.120503,0.0
4,165236,2.180149,0.218732,-2.637726,0.348776,1.063546,-1.249197,0.942021,-0.547652,-0.087823,...,-0.250653,0.234502,0.825237,-0.176957,0.563779,0.730183,0.707494,-0.131066,-0.090428,0.0
5,62606,1.199408,0.352007,0.379645,1.372017,0.291347,0.524919,-0.117555,0.132907,-0.935169,...,-0.042979,-0.050291,-0.126609,-0.022218,-0.599026,0.258188,0.928721,-0.058988,-0.008856,0.0
6,90719,1.937447,0.337882,-0.000630,3.816486,0.276515,1.079842,-0.730626,0.197353,1.137566,...,-0.315667,-0.038376,0.208914,0.160189,-0.015145,-0.162678,-0.000843,-0.018178,-0.039339,0.0
7,113350,1.891900,0.401086,-0.119983,4.047500,0.049952,0.192793,-0.108512,-0.040400,-0.390391,...,-0.267639,0.094177,0.613712,0.070986,0.079543,0.135219,0.128961,0.003667,-0.045079,0.0
8,156499,0.060003,1.461355,0.378915,2.835455,1.626526,-0.164732,1.551858,-0.412927,-1.735264,...,-0.175275,0.042293,0.277536,-0.123379,1.081552,-0.053079,-0.149809,-0.314438,-0.216539,0.0
9,73902,-1.859260,2.158799,1.085671,2.615483,0.246660,2.133925,-1.569015,-2.612353,-1.312509,...,0.590142,-0.867178,-0.700479,0.231972,-1.374527,0.140285,0.128806,0.153606,0.092042,0.0


In [54]:
newobs = pred.to_dict(orient = 'records')
#newobs[0]

In [55]:
newobs = [{k:[v] for k,v in newob.items()} for newob in newobs]

In [56]:
newobs[0]

{'Time': [35337],
 'V1': [1.0928441854981998],
 'V2': [-0.0132303486713432],
 'V3': [1.35982868199426],
 'V4': [2.7315370965921004],
 'V5': [-0.707357349219652],
 'V6': [0.8738370029866129],
 'V7': [-0.7961301510622031],
 'V8': [0.437706509544851],
 'V9': [0.39676985012996396],
 'V10': [0.587438102569443],
 'V11': [-0.14979756231827498],
 'V12': [0.29514781622888103],
 'V13': [-1.30382621882143],
 'V14': [-0.31782283120234495],
 'V15': [-2.03673231037199],
 'V16': [0.376090905274179],
 'V17': [-0.30040350116459497],
 'V18': [0.433799615590844],
 'V19': [-0.145082264348681],
 'V20': [-0.240427548108996],
 'V21': [0.0376030733329398],
 'V22': [0.38002620963091405],
 'V23': [-0.16764742731151097],
 'V24': [0.0275573495476881],
 'V25': [0.59211469704354],
 'V26': [0.219695164116351],
 'V27': [0.0369695108704894],
 'V28': [0.010984441006191],
 'Amount': [0.0]}

### Get Predictions: Python Client

In [57]:
prediction = endpoint.predict(instances = newobs[0:1])
prediction

Prediction(predictions=[[0.999759495, 0.000240550202]], deployed_model_id='2423682068408958976', metadata=None, model_version_id='18', model_resource_name='projects/1026793852137/locations/us-central1/models/model_05_05', explanations=None)

In [58]:
prediction = endpoint.predict(instances = newobs)
prediction

Prediction(predictions=[[0.999759436, 0.000240550187], [0.999389887, 0.000610154064], [0.999763489, 0.000236547974], [0.999877691, 0.000122421377], [0.999873161, 0.000126962157], [0.999579489, 0.000420496828], [0.999929368, 7.053015e-05], [0.999938548, 6.13750744e-05], [0.999892116, 0.00010785809], [0.999943733, 5.62981877e-05]], deployed_model_id='2423682068408958976', metadata=None, model_version_id='18', model_resource_name='projects/1026793852137/locations/us-central1/models/model_05_05', explanations=None)

In [59]:
prediction.predictions[0]

[0.999759436, 0.000240550187]

In [60]:
np.argmax(prediction.predictions[0])

0

### Get Predictions: REST

In [61]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": newobs[0:1]}))

In [62]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    [
      0.999759495,
      0.000240550202
    ]
  ],
  "deployedModelId": "2423682068408958976",
  "model": "projects/1026793852137/locations/us-central1/models/model_05_05",
  "modelDisplayName": "05_05",
  "modelVersionId": "18"
}


### Get Predictions: gcloud (CLI)

In [63]:
!gcloud ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[[0.999759495, 0.000240550202]]


---
## Pipeline: Continous Training & Serving

Create a pipeline job on Vertex AI Pipelines that does the complete workflow above: setup, train, register model, rank model, bless model (compare to currently deployed model), update endpoint, test endpoint.

First, read about pipelines [here](../MLOps.md#Pipelines)!

This section builds a [Kubeflow Pipeline](https://www.kubeflow.org/docs/components/pipelines/v1/introduction/) and runs it on [Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction).

In [ ]:
import kfp
from google_cloud_pipeline_components.types import artifact_types
from typing import NamedTuple

In [ ]:
kfp.__version__

### Store Training Code In GCS

This workflow uses [aiplatform.CustomJob.from_local_script()](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.CustomJob#google_cloud_aiplatform_CustomJob_from_local_script) to create a [Vertex AI Training Custom Job](https://cloud.google.com/vertex-ai/docs/training/create-custom-job#create_custom_job-python_vertex_ai_sdk).  The key is the method expects the file to be local to the request.  When a Vertex AI Pipeline job runs, the component that is built below will execute this method.  Saving the file to a known GCS location means that the component can reference it as local by using the automatically created [GCS Storage FUSE](https://cloud.google.com/vertex-ai/docs/training/cloud-storage-file-system) mounted at the location `/gcs/<bucket-name>/path-to-file.ext`.

In [ ]:
bucket = gcs.lookup_bucket(GCS_BUCKET)

In [ ]:
SCRIPT_PATH

In [ ]:
SCRIPT_NAME = SCRIPT_PATH.split('/')[-1]
SCRIPT_NAME

In [ ]:
blob = bucket.blob(f'{SERIES}/{EXPERIMENT}/training/{SCRIPT_NAME}')

In [ ]:
blob.upload_from_filename(SCRIPT_PATH)

### Component: experiment_setup

The first component in the pipeline.  It makes sure the `experiment` and `series` exist and sets up a Tensorboard instance if needed.  It also create a `run_name` for this run of the `experiment` while also adding thes variables to the `CMDARGS` that are input to the training job.  It even gets a single instance from the test data to use for prediction test during the pipeline execution.

In [ ]:
@kfp.dsl.component(
    base_image = "python:3.9",
    packages_to_install = ["google-cloud-aiplatform", "google-cloud-pipeline-components", "google-cloud-bigquery", "pandas", "db-dtypes"]
)
def experiment_setup(
    project: str,
    region: str,
    series: str,
    experiment: str,
    experiment_name: str,
    cmdargs: list,
    bq_project: str,
    bq_dataset: str,
    bq_table: str,
    var_target: str,
    var_omit: str,
    bq_source: kfp.dsl.Output[artifact_types.BQTable]
) -> NamedTuple('outputs', [
    ('cmdargs', list), 
    ('run_name', str), 
    ('tensorboard', str), 
    ('timestamp', str),
    ('experiment_uri', str),
    ('experiment_run_uri', str),
    ('sample', dict)]):

    # setup the output parameters
    from collections import namedtuple
    result = namedtuple('outputs', ['cmdargs', 'run_name', 'tensorboard', 'timestamp', 'experiment_uri', 'experiment_run_uri', 'sample'])
    
    # vertex client
    from google.cloud import aiplatform
    aiplatform.init(project = project, location = region)

    # bigquery client
    from google.cloud import bigquery
    bq = bigquery.Client(project = project)
    
    # return/create TensorBoard instance for series
    tb = aiplatform.Tensorboard.list(filter=f"labels.series={series}")
    if tb:
        tb = tb[0]
    else: 
        tb = aiplatform.Tensorboard.create(display_name = series, labels = {'series' : f'{series}'})
    
    # initialize experiment - in case it is new
    aiplatform.init(experiment = experiment_name, experiment_tensorboard = tb.resource_name)
    
    # Output Parameters: update parameters list
    from datetime import datetime
    TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
    RUN_NAME = f'run-{TIMESTAMP}'
    CMDARGS = cmdargs + [
        "--project_id=" + project,
        "--region=" + region,
        "--series=" + series,
        "--experiment=" + experiment,
        "--experiment_name=" + experiment_name,
        "--run_name=" + RUN_NAME,
        "--bq_project=" + bq_project,
        "--bq_dataset=" + bq_dataset,
        "--bq_table=" + bq_table,
        "--var_target=" + var_target,
        "--var_omit=" + var_omit
    ]
    
    # Output Parameter: get sample for prediction
    OMIT = [x for x in var_omit.split(',') if x != '']
    samples = bq.query(query = f"""
        SELECT * EXCEPT({','.join([var_target] + OMIT)})
        FROM `{bq_project}.{bq_dataset}.{bq_table}`
        WHERE splits = 'TEST'
        LIMIT 1
    """).to_dataframe().to_dict(orient = 'records')
    
    # Output Artifact: BigQuery Table 
    bq_source.uri = f'https://www.googleapis.com/bigquery/v2/projects/{bq_project}/datasets/{bq_dataset}/tables/{bq_table}'
    bq_source.metadata['projectId'] = bq_project
    bq_source.metadata['datasetId'] = bq_dataset
    bq_source.metadata['tableId'] = bq_table

    # create links for the experiment and experiment run
    exp_uri = f'https://console.cloud.google.com/vertex-ai/locations/{region}/experiments/{experiment_name}?project={project}'
    exp_run_uri = f'https://console.cloud.google.com/vertex-ai/locations/{region}/experiments/{experiment_name}/runs/{experiment_name}-{RUN_NAME}?project={project}'

    # return the output parameters
    return result(CMDARGS, RUN_NAME, tb.resource_name, TIMESTAMP, exp_uri, exp_run_uri, samples[0])

### Component: train_from_local_script

Receives the pipeline inputs as well as the runs setup information from the `experiment_setup` component defined above.  This component is used to launch the Vertex AI Custom training job and wait for its completion. It uses the local `/gcs` folder, a GCS Fuse mount, to pass the script as though it is local.  The fuse mount is an automatically created [GCS Storage FUSE](https://cloud.google.com/vertex-ai/docs/training/cloud-storage-file-system) mounted at the location `/gcs/<bucket-name>/path-to-file.ext`.

This uses a special type of parameter `gcp_resource` that renders the job information in the console for review while training is running. [Reference](https://cloud.google.com/vertex-ai/docs/pipelines/build-own-components)

In [ ]:
@kfp.dsl.component(
    base_image = "python:3.9",
    packages_to_install = ["google-cloud-aiplatform", "google-cloud-pipeline-components"]
)
def train_from_local_script(
    project: str,
    region: str,
    series: str,
    experiment: str,
    experiment_name: str,
    run_name: str,
    cmdargs: list,
    script: str,
    train_image: str,
    requirements: list,
    train_compute: str,
    timestamp: str,
    bucket: str,
    service_account: str,
    tensorboard: str,
    job_resources: kfp.dsl.Output[kfp.dsl.Artifact]
) -> NamedTuple('outputs', [('state', str), ('gcp_resources', str)]):
    
    # setup the output parameters
    from collections import namedtuple
    result = namedtuple('outputs', ['state', 'gcp_resources'])
    
    # vertex client
    from google.cloud import aiplatform
    aiplatform.init(project = project, location = region)
    
    # create job:
    customJob = aiplatform.CustomJob.from_local_script(
        display_name = f'{series}_{experiment}_{timestamp}',
        script_path = f'/gcs/{bucket}/{series}/{experiment}/training/{script}',
        container_uri = train_image,
        args = cmdargs,
        requirements = requirements,
        replica_count = 1,
        machine_type = train_compute,
        accelerator_count = 0,
        base_output_dir = f"gs://{bucket}/{series}/{experiment}/models/{timestamp}",
        staging_bucket = f"gs://{bucket}/{series}/{experiment}/models/{timestamp}",
        labels = {'series' : f'{series}', 'experiment' : f'{experiment}', 'experiment_name' : f'{experiment_name}', 'run_name' : f'{run_name}'}
    )    

    # run job:
    customJob.run(
        service_account = service_account,
        tensorboard = tensorboard
    )

    # Output Artifact: job resources
    job_resources.uri = f"https://console.cloud.google.com/vertex-ai/locations/{region}/training/{customJob.resource_name.split('/')[-1]}/cpu"
    job_resources.metadata = dict(state = customJob.state.name, resource_name = customJob.resource_name, model = f"gs://{bucket}/{series}/{experiment}/models/{timestamp}/model")
    
    # Output Special: gcp_resource parameter for in concole monitoring of the job
    # info: https://cloud.google.com/vertex-ai/docs/pipelines/build-own-components
    # GCP Resource Proto: https://github.com/kubeflow/pipelines/tree/master/components/google-cloud/google_cloud_pipeline_components/proto
    from google_cloud_pipeline_components.proto.gcp_resources_pb2 import GcpResources
    from google.protobuf.json_format import MessageToJson
    customJob_resources = GcpResources()
    cr = customJob_resources.resources.add()
    cr.resource_type = 'CustomJob'
    # customJob URI like: https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.customJobs/create
    cr.resource_uri = f'https://{region}-aiplatform.googleapis.com/v1/{customJob.resource_name}'
    gcp_resources = MessageToJson(customJob_resources)
    
    return result(customJob.state.name, gcp_resources)

### Component: register_model

Register the resulting model as a new version and update the experiment data with model and job information.

Output Model Artifact:
- use Vertex AI Model Artifact: https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-2.0.0/api/artifact_types.html#
    - GitHub Source: https://github.com/kubeflow/pipelines/blob/master/components/google-cloud/google_cloud_pipeline_components/types/artifact_types.py

In [ ]:
@kfp.dsl.component(
    base_image = "python:3.9",
    packages_to_install = ["google-cloud-aiplatform", "google-cloud-pipeline-components"]
)
def register_model(
    project: str,
    region: str,
    series: str,
    experiment: str,
    experiment_name: str,
    run_name: str,
    deploy_image: str,
    job_resources: kfp.dsl.Input[kfp.dsl.Artifact],
    tensorboard: str,
    vertex_model: kfp.dsl.Output[artifact_types.VertexModel]
) -> NamedTuple('outputs', [('version_id', str)]):
    
    # setup the output parameters
    from collections import namedtuple
    result = namedtuple('outputs', ['version_id'])
    
    # vertex client
    from google.cloud import aiplatform
    aiplatform.init(project = project, location = region)
    
    # Upload Model
    modelmatch = aiplatform.Model.list(filter = f'display_name={series}_{experiment} AND labels.series={series} AND labels.experiment={experiment}')

    upload_model = True
    if modelmatch:
        print("Model Already in Registry:")
        if run_name in modelmatch[0].version_aliases:
            print("This version already loaded, no action taken.")
            upload_model = False
            model = aiplatform.Model(model_name = modelmatch[0].resource_name)
        else:
            print('Loading model as new default version.')
            parent_model = modelmatch[0].resource_name

    else:
        print('This is a new model, creating in model registry')
        parent_model = ''

    if upload_model:
        model = aiplatform.Model.upload(
            display_name = f'{series}_{experiment}',
            model_id = f'model_{series}_{experiment}',
            parent_model =  parent_model,
            serving_container_image_uri = deploy_image,
            artifact_uri = job_resources.metadata['model'], 
            is_default_version = True,
            version_aliases = [run_name],
            version_description = run_name,
            labels = {'series' : f'{series}', 'experiment' : f'{experiment}', 'experiment_name' : f'{experiment_name}', 'run_name' : f'{run_name}'}        
        )

    #print(f'Review the model in the Vertex AI Model Registry:\nhttps://console.cloud.google.com/vertex-ai/locations/{region}/models/{model.name}?project={project}')

    customJob = aiplatform.CustomJob.get(resource_name = job_resources.metadata['resource_name'])
    board_link = f"https://{region}.tensorboard.googleusercontent.com/experiment/{tensorboard.replace('/', '+')}+experiments+{customJob.resource_name.split('/')[-1]}"
    
    
    expRun = aiplatform.ExperimentRun(run_name = run_name, experiment = experiment_name)
    expRun.log_params({
        'model.uri': model.uri,
        'model.display_name': model.display_name,
        'model.name': model.name,
        'model.resource_name': model.resource_name,
        'model.version_id': model.version_id,
        'model.versioned_resource_name': model.versioned_resource_name,
        'customJobs.display_name': customJob.display_name,
        'customJobs.resource_name': customJob.resource_name,
        'customJobs.link': job_resources.uri,
        'customJobs.tensorboard': board_link
    })

    expRun.update_state(state = aiplatform.gapic.Execution.State.COMPLETE)

    #print(f'Review The Experiment in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{region}/experiments/{experiment_name}?project={project}')
    #print(f'Review The Experiment Run in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{region}/experiments/{experiment_name}/runs/{experiment_name}-{run_name}?project={project}')

    # Output Artifact: Vertex Model (versioned)
    #vertex_model.name = model.display_name
    vertex_model.uri = f'https://{region}-aiplatform.googleapis.com/v1/{model.versioned_resource_name}'
    vertex_model.metadata['model_resource_name'] = model.versioned_resource_name
    
    return result(model.version_id)

## Component: endpoint_model

Retrieve (or Create) the endpoint and get the currently deployed model receiving traffic.

Ouput Artifacts for the endpoint and current model on endpoint:
- use Vertex AI Model Artifact: https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-2.0.0/api/artifact_types.html#
    - GitHub Source: https://github.com/kubeflow/pipelines/blob/master/components/google-cloud/google_cloud_pipeline_components/types/artifact_types.py

In [ ]:
@kfp.dsl.component(
    base_image = "python:3.9",
    packages_to_install = ["google-cloud-aiplatform", "google-cloud-pipeline-components", 'pandas']
)
def endpoint_model(
    project: str,
    region: str,
    series: str,
    vertex_endpoint: kfp.dsl.Output[artifact_types.VertexEndpoint],
    vertex_model: kfp.dsl.Output[artifact_types.VertexModel]
):
    
    # vertex client
    from google.cloud import aiplatform
    aiplatform.init(project = project, location = region)
    
    endpoints = aiplatform.Endpoint.list(filter = f"labels.series={series} AND display_name={series}")
    if endpoints:
        endpoint = endpoints[0]
        print(f"Endpoint Exists: {endpoints[0].resource_name}")
    else:
        endpoint = aiplatform.Endpoint.create(
            display_name = f"{series}",
            labels = {'series' : f"{series}"}    
        )
        print(f"Endpoint Created: {endpoint.resource_name}")

    # Output Artifact: Vertex Endpoint
    vertex_endpoint.uri = f'https://{region}-aiplatform.googleapis.com/v1/{endpoint.resource_name}'
    vertex_endpoint.metadata['endpoint_resource_name'] = endpoint.resource_name
    
    # Output Artifact: Vertex Model
    for model in endpoint.list_models():
        if endpoint.traffic_split[model.id] == 100:
            vertex_model.uri = f'https://{region}-aiplatform.googleapis.com/v1/{model.model}@{model.model_version_id}'
            vertex_model.metadata['model_resource_name'] = model.model+'@'+model.model_version_id
    print(model.model_version_id)

### Component: rank_model

Use Vertex AI Experiments to retrieve information from all runs in this experiment and all experiments in this series.  Then rank models and versions.  Use the ranks to compare the newly trained model to the model currently on the endpoint.  Ouput the ranks table as a [Markdown Artifact](https://kubeflow-pipelines.readthedocs.io/en/latest/source/dsl.html#kfp.dsl.Markdown) that is reviewable in the console for the pipeline.

In [ ]:
@kfp.dsl.component(
    base_image = "python:3.9",
    packages_to_install = ["google-cloud-aiplatform", "google-cloud-pipeline-components", 'pandas']
)
def rank_model(
    project: str,
    region: str,
    series: str,
    current_model: kfp.dsl.Input[artifact_types.VertexModel], 
    new_model: kfp.dsl.Input[artifact_types.VertexModel],
    ranks_table: kfp.dsl.Output[kfp.dsl.Markdown]
) -> NamedTuple('outputs', [('new_model_experiment_rank', float), ('new_model_series_rank', float), ('current_model_series_rank', float)]):
    
    # setup the output parameters
    from collections import namedtuple
    result = namedtuple('outputs', ['new_model_experiment_rank', 'new_model_series_rank', 'current_model_series_rank'])
    
    # vertex client
    from google.cloud import aiplatform
    aiplatform.init(project = project, location = region)

    import pandas as pd
    
    # get models
    current_model = aiplatform.Model(model_name = current_model.metadata['model_resource_name'])
    new_model = aiplatform.Model(model_name = new_model.metadata['model_resource_name'])
    
    # list experiments in project
    experiments = aiplatform.Experiment.list()
    # filter to experiment is SERIES
    experiments = [e for e in experiments if e.name.split('-')[0:2] == ['experiment', series]]
    # combine runs from all experiment in SERIES into single dataframe
    results = []
    for experiment in experiments:
            results.append(experiment.get_data_frame())
            print(experiment.name)
    results = pd.concat(results)
    # define rank for models: within experiment and across entire series
    def ranker(metric = 'metric.test_auprc'):
        ranks = results[['experiment_name', 'run_name', 'param.model.display_name', 'param.model.version_id', metric]].copy().reset_index(drop = True)
        ranks = ranks[ranks['param.model.display_name'].notnull()]
        ranks['series_rank'] = ranks[metric].rank(method = 'dense', ascending = False)
        ranks['experiment_rank'] = ranks.groupby('experiment_name')[metric].rank(method = 'dense', ascending = False)
        return ranks.sort_values(by = ['series_rank', 'experiment_rank'])
    ranks = ranker('metric.test_auprc') 
    # mark current model and new model:
    ranks.loc[(ranks['param.model.display_name'] == current_model.display_name) & (ranks['param.model.version_id'] == current_model.version_id), 'Status'] = 'On Endpoint'
    ranks.loc[(ranks['param.model.display_name'] == new_model.display_name) & (ranks['param.model.version_id'] == new_model.version_id), 'Status'] = 'New Model'
    col = ranks.pop('Status')
    ranks.insert(0, col.name, col)
    # get value for current and new model:
    current_rank = ranks.loc[(ranks['param.model.display_name'] == current_model.display_name) & (ranks['param.model.version_id'] == current_model.version_id)]
    new_rank = ranks.loc[(ranks['param.model.display_name'] == new_model.display_name) & (ranks['param.model.version_id'] == new_model.version_id)]
    
    # output markdown artifact: the ranks table
    with open(ranks_table.path, 'w') as f:
        f.write(ranks.to_markdown(index = False))
    
    return result(new_rank['experiment_rank'].iloc[0], new_rank['series_rank'].iloc[0], current_rank['series_rank'].iloc[0])

### Component: deploy_model

A component that deploys the new model to the endpoint and removes traffic from older models on the endpoint, and undeploys them.  Returns the status of the deployment and confirmed traffic percentage.

In [ ]:
@kfp.dsl.component(
    base_image = "python:3.9",
    packages_to_install = ["google-cloud-aiplatform", "google-cloud-pipeline-components"]
)
def deploy_model(
    project: str,
    region: str,
    deploy_compute: str,
    vertex_endpoint: kfp.dsl.Input[artifact_types.VertexEndpoint], 
    vertex_model: kfp.dsl.Input[artifact_types.VertexModel]
) -> NamedTuple('outputs', [('count_of_deployed', int), ('new_model_deployed', bool), ('traffic_new_model', int)]):

    # setup the output parameters
    from collections import namedtuple
    result = namedtuple('outputs', ['count_of_deployed', 'new_model_deployed', 'traffic_new_model'])
    
    # vertex client
    from google.cloud import aiplatform
    aiplatform.init(project = project, location = region)
    
    endpoint = aiplatform.Endpoint(endpoint_name = vertex_endpoint.metadata['endpoint_resource_name'])
    model = aiplatform.Model(model_name = vertex_model.metadata['model_resource_name'])
    
    # deploy the model to the endpoint
    endpoint.deploy(
        model = model,
        deployed_model_display_name = model.display_name,
        traffic_percentage = 100,
        machine_type = deploy_compute,
        min_replica_count = 1,
        max_replica_count = 1
    )

    # remove models without traffic
    deployed = False
    traffic = 0
    for deployed_model in endpoint.list_models():
        if deployed_model.id not in endpoint.traffic_split:
            endpoint.undeploy(deployed_model_id = deployed_model.id)
        elif f'{deployed_model.model}@{deployed_model.model_version_id}' == vertex_model.metadata['model_resource_name']:
            deployed = True
            traffic = endpoint.traffic_split[deployed_model.id]

    info = endpoint.to_dict()
    
    return result(
        len(info['deployedModels']),
        deployed,
        traffic
    )

### Component: prediction_test

A custom component to run a prediction request on the endpoint (input artifact) and return the result of the prediction.

In [ ]:
@kfp.dsl.component(
    base_image = "python:3.9",
    packages_to_install = ["google-cloud-aiplatform", "google-cloud-pipeline-components"]
)
def prediction_test(
    project: str,
    region: str,
    sample: dict,
    vertex_endpoint: kfp.dsl.Input[artifact_types.VertexEndpoint]
) -> NamedTuple('outputs', [('predictions', list), ('model_resource_name', str)]):

    # setup the output parameters
    from collections import namedtuple
    result = namedtuple('outputs', ['predictions', 'model_resource_name'])

    
    # vertex client
    from google.cloud import aiplatform
    aiplatform.init(project = project, location = region) 
    
    endpoint = aiplatform.Endpoint(endpoint_name = vertex_endpoint.metadata['endpoint_resource_name'])
    
    prediction = endpoint.predict(instances = [sample])
    
    return result(
        predictions = prediction.predictions[0],
        model_resource_name = prediction.model_resource_name + '@' + prediction.model_version_id
    )

### Pipeline

Create the pipeline using [kfp.dsl.pipeline()](https://kubeflow-pipelines.readthedocs.io/en/latest/source/dsl.html#kfp.dsl.pipeline) using features like:
- conditional execution for registering model and deploying the new model to the endpoint with [kfp.dsl.Condition](https://kubeflow-pipelines.readthedocs.io/en/latest/source/dsl.html#kfp.dsl.Condition)
- naming components with `.set_display_name()`
- controling caching of components with `.set_caching_options()`
- set [compute resources for a components VM](https://cloud.google.com/vertex-ai/docs/pipelines/machine-types) with `.set_cpu_limit()`
- force order of exectuion with `.after()`


In [ ]:
@kfp.dsl.pipeline(
    name = f'series-{SERIES}-{EXPERIMENT}-pipeline',
    description = 'Full development pipeline.'
)
def retrain_pipeline(
    project: str,
    region: str,
    series: str,
    experiment: str,
    experiment_name: str,
    bq_project: str,
    bq_dataset: str,
    bq_table: str,
    var_target: str,
    var_omit: str,
    cmdargs: list,
    script: str,
    train_image: str,
    requirements: list,
    train_compute: str,
    deploy_image: str,
    deploy_compute: str,
    bucket: str,
    service_account: str
):

    from google_cloud_pipeline_components.types import artifact_types
    
    # run setup
    setup = experiment_setup(
        project = project,
        region = region,
        series = series,
        experiment = experiment,
        experiment_name = experiment_name,
        cmdargs = cmdargs,
        bq_project = bq_project,
        bq_dataset = bq_dataset,
        bq_table = bq_table,
        var_target = var_target,
        var_omit = var_omit
    ).set_display_name('Setup').set_cpu_limit('2').set_caching_options(True)
    
    # launch training job from component - using script stored in GCS
    trainer = train_from_local_script(
        project = project,
        region = region,
        series = series,
        experiment = experiment,
        experiment_name = experiment_name,
        run_name = setup.outputs['run_name'],
        cmdargs = setup.outputs['cmdargs'],
        script = script,
        train_image = train_image,
        requirements = requirements,
        train_compute = train_compute,
        timestamp = setup.outputs['timestamp'],
        bucket = bucket,
        service_account = service_account,
        tensorboard = setup.outputs['tensorboard']
    ).set_display_name('Train With Script').set_cpu_limit('2').set_caching_options(True)
    
    
    # check if train was sucessful
    with kfp.dsl.If(
        trainer.outputs['state'] == 'JOB_STATE_SUCCEEDED',
        name = 'Check Training Completion'
    ):
    
        endpoint_before = endpoint_model(
            project = project,
            region = region,
            series = series
        ).set_display_name('Endpoint: Before Update').set_cpu_limit('2').set_caching_options(False)
        
        model = register_model(
            project = project,
            region = region,
            series = series,
            experiment = experiment,
            experiment_name = experiment_name,
            run_name = setup.outputs['run_name'],
            deploy_image = deploy_image,
            job_resources = trainer.outputs['job_resources'],
            tensorboard = setup.outputs['tensorboard']
        ).set_display_name('Register Model').set_cpu_limit('2').set_caching_options(False)
    
        rank = rank_model(
            project = project,
            region = region,
            series = series,
            new_model = model.outputs['vertex_model'],
            current_model = endpoint_before.outputs['vertex_model']
        ).set_display_name('Rank Model').set_cpu_limit('2').set_caching_options(False)
    
        with kfp.dsl.If(
            rank.outputs['new_model_series_rank'] < rank.outputs['current_model_series_rank'],
            name = 'Bless New Model'
        ):
            
            deploy = deploy_model(
                project = project,
                region = region,
                deploy_compute = deploy_compute,
                vertex_endpoint = endpoint_before.outputs['vertex_endpoint'],
                vertex_model = model.outputs['vertex_model']
            ).set_display_name('Deploy Model').set_cpu_limit('2').set_caching_options(False)
            
            endpoint_after = endpoint_model(
                project = project,
                region = region,
                series = series
            ).set_display_name('Endpoint: After Update').set_cpu_limit('2').set_caching_options(False).after(deploy)
    
            with kfp.dsl.If(
                deploy.outputs['new_model_deployed'] == True,
                name = 'Check The Model Deployment'
            ):
    
                prediction = prediction_test(
                    project = project,
                    region = region,
                    sample = setup.outputs['sample'],
                    vertex_endpoint = endpoint_after.outputs['vertex_endpoint'],
                ).set_display_name('Prediction').set_cpu_limit('2').set_caching_options(False)

### Compile Pipeline

Compile the pipeline using [kfp.compiler.Compiler()](https://kubeflow-pipelines.readthedocs.io/en/latest/source/compiler.html#kfp.compiler.Compiler).  Here the `json` format is used but alternatively the `yaml` format could be used for storing this as a [pipeline template in artifact registry](https://cloud.google.com/vertex-ai/docs/pipelines/create-pipeline-template).

In [ ]:
kfp.compiler.Compiler().compile(
    pipeline_func = retrain_pipeline,
    package_path = f"{DIR}/{EXPERIMENT}.json"
)

### Define Pipeline Job

Setup the input parameters for the pipeline and create a pipeline job with the SDK [aiplatform.PipelineJob()](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.PipelineJob).

In [ ]:
REQUIREMENTS = ['tensorflow_io', f'google-cloud-aiplatform>={aiplatform.__version__}', 'db-dtypes', f"protobuf>={importlib.metadata.version('protobuf')}"]

In [ ]:
CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--batch_size=" + str(BATCH_SIZE)
]

In [ ]:
parameter_values = {
    "project": PROJECT_ID,
    "region": REGION,
    "series": SERIES,
    "experiment": EXPERIMENT,
    "experiment_name": EXPERIMENT_NAME,
    "bq_project": BQ_PROJECT,
    "bq_dataset": BQ_DATASET,
    "bq_table": BQ_TABLE,
    "var_target": VAR_TARGET,
    "var_omit": VAR_OMIT,
    "cmdargs": CMDARGS,
    "script": SCRIPT_NAME,
    "train_image": TRAIN_IMAGE,
    "requirements": REQUIREMENTS,
    "train_compute": TRAIN_COMPUTE,
    "deploy_image": DEPLOY_IMAGE,
    "deploy_compute": DEPLOY_COMPUTE,
    "bucket": GCS_BUCKET,
    "service_account": SERVICE_ACCOUNT
}

In [ ]:
pipeline_job = aiplatform.PipelineJob(
    display_name = f"{SERIES}_{EXPERIMENT}",
    template_path = f"{DIR}/{EXPERIMENT}.json",
    parameter_values = parameter_values,
    pipeline_root = f"{URI}/pipeline_root",
    enable_caching = None, # True (enabled), False (disable), None (defer to component level caching) 
    labels = {'series': SERIES, 'experiment': EXPERIMENT}
)

### Submit Pipeline Job

Submit the pipeline job for execution with [aiplatform.PipelineJob.sumit()](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.PipelineJob#google_cloud_aiplatform_PipelineJob_submit).

In [ ]:
response = pipeline_job.submit(
    service_account = SERVICE_ACCOUNT
)

In [ ]:
print(f'The Dashboard can be viewed here:\n{pipeline_job._dashboard_uri()}')

### Wait On Pipeline Job

For the execution to wait on the completion of the pipeline job with [aiplatform.PipelineJob.wait()](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.PipelineJob#google_cloud_aiplatform_PipelineJob_wait).

In [ ]:
pipeline_job.wait()

### Retrieve Pipeline Information

Retrive the pipeline execution information for all runs of this pipeline (have the same name) using [aiplatform.get_pipeline_df()](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_get_pipeline_df).

In [ ]:
aiplatform.get_pipeline_df(pipeline = f'series-{SERIES}-{EXPERIMENT}-pipeline')

In [ ]:
tasks = {task.task_name: task for task in pipeline_job.task_details}

In [ ]:
for task in tasks:
  print(task, tasks[task].state)

In [ ]:
tasks.get('rank-model')

---
## Monitor Pipeline: In Console

This is what the pipeline looks like in the Vertex AI Console:

<p align="center" width="90%">
    <img src="../architectures/notebooks/05/05a_pipeline.png" width="75%">
</p>

## TODO
- how to handle when endpoint does not have current model - new endpoint?
- rework manual workflow ranks section to match the new component
- rework experiments integration
- add model evaluation to registry
- save pipeline to artifact registry - show in Vertex AI Pipeline Templates
- Schedule pipeline execution: from pipeline or artifact registry
- include resource removal in notebook

---
## Remove Resources
see notebook "99 - Cleanup"